In [2]:
import cirq

We are now going to explore the quantum Fourier transformation, which is an important ingredient for many quantum algorithms with superpolynomial speed-up. 

Rading Ch 7.1, RC 19642 (07/12/94) (arXiv:quant-ph/0201067)


Before specifying the algorithm, we want to define the desired outcome and also establish some notation.
We define the Quantum Fourier Transform (QFT) with respect to the basis states $\{|0\rangle\},|1\rangle, ..., |m-1\rangle$, where we now understand $|y\rangle$ as the state corresponding to state $|\mathbf{y}\rangle$ where $\mathbf{y}$ is the binary encoding of $y$, in the following way
\begin{definition}
\begin{equation}
\mathrm{QFT}_m: |x\rangle \rightarrow \frac{1}{\sqrt{m}}\sum^{m-1}_{y=0}e^{2\pi i \frac{x}{m} y }|y\rangle
\end{equation}
\end{definition}
Here we understand $xy$ as normal integer multiplication. In order to understand how this translates to the binary setting of qubit system let us also sepcify, how we understand a general phase $\omega \in [0,1)$. Here we view $\omega$ as written in binary, i.e. $\omega = 0.x_1x_2x_3... = \sum 2^{-i}x_i $. This immediedtly leads to the identity 
\begin{equation}
e^{2\pi i 2^k \omega} =e^{2\pi i  x_1x_2...x_k.x_{k+1}...} = e^{0.x_{k+1}x_{k+2}...}
\end{equation}
Lastly we want to state a useful identity, which also hopefully clearifies the notation further. Before we state the identity we note that $\sum^n_{k=0} 2^k = 2^n-1$ and note that to encode numbers up to $2^{n-1}$ in binary we need $n$-bits. This allows us now for example to write $e^{2\pi i \omega 2^n-1}| 2^n-1\rangle = e^{2\pi i \omega 2^n-1}|1,1,...1\rangle = (e^{2\pi i \omega 2^{n-1}}|1\rangle) \otimes  (e^{2\pi i \omega 2^{n-2}}|1\rangle) \otimes...(e^{2\pi i \omega}|1\rangle) $ in the same way we can now rewrite all states as products, (just to illustrate the point a bit further $e^{2\pi i \omega 2^n-2}| 2^n-2\rangle =  (e^{2\pi i \omega 2^{n-1}}|1\rangle) \otimes  (e^{2\pi i \omega 2^{n-2}}|1\rangle) \otimes...|0\rangle)$) which then immediatley leads to the identity
\begin{equation}
\frac{1}{\sqrt{2}}\sum^{2^n-1}_{y = 0} e^{2\pi i \omega y }|y\rangle =  \frac{1}{\sqrt{2}} \left(|0\rangle +(e^{2\pi i \omega 2^{n-1}}|1\rangle)\right)\otimes\left(|0\rangle +(e^{2\pi i \omega 2^{n-2}}|1\rangle)\right)\otimes...\otimes \left(|0\rangle +e^{2\pi i \omega}|1\rangle\right)
\end{equation}
this can now be further simplified using (2).

\begin{equation}
\frac{1}{\sqrt{2}}\sum^{2^n-1}_{y = 0} e^{2\pi i \omega y }|y\rangle =\frac{1}{\sqrt{2^n}} \left(|0\rangle+e^{2\pi i 0.x_nx_{n+1}...}|1\rangle \right)\otimes \left(|0\rangle+e^{2\pi i 0.x_{n-1}x_{n}...}|1\rangle \right)\otimes...\otimes \left(|0\rangle+e^{2\pi i 0.x_{1}x_{2}...}|1\rangle \right)
\end{equation}
We can now see that the QFT in (1) can be understood as the direct product in (4). We can now devise a strategy to implement the QFT. If we start out with n registers in the state $|x\rangle = |x_1x_2...x_n\rangle =|x_1\rangle \otimes |x_2\rangle \otimes...\otimes |x_n\rangle $ we want to build the circuit in such a way, that $|x_i\rangle \rightarrow \frac{1}{\sqrt{2}}\big(|0\rangle + e^{2\pi i 0.x_ix_{i+1},...x_n}|1\rangle\big)$, which then in turn gives (4)

Before turning to the details of the curcuit implementing the QFT we want to introduced the controlled phase rotation operator, which is defined as 
\begin{equation}
cR_\phi = |0\rangle\langle 0| \otimes I + |0\rangle\langle 0| \otimes \begin{bmatrix}
1 & 0\\
0 & e^{i\phi} 
\end{bmatrix}	
\end{equation}
where  $R_\phi$ is also expressed in the computational basis. We will write $R_n$ to refer to $\phi= 2^{-n}$

We are now able to devise a strategy for implementing the $QFT_{2^n}$. We start by applying a Hadamard gate to every register, which leads to 
\begin{equation}
H |x_i\rangle = \frac{1}{\sqrt{2}} \left(|0\rangle + (-1)^{x_i}\right|1\rangle) = \frac{1}{\sqrt{2}} \left(|0\rangle + e^{\pi i x_i}\right|1\rangle) = \frac{1}{\sqrt{2}} \left(|0\rangle + e^{2\pi i 0.x_i}\right|1\rangle)
\end{equation}
This shows that we already have the right first element for each phase. If we now apply a decreasing controlled phase gates (i.e. $cR_2, \, cR_3,\, ...,cR_{n-{i+1}}$) controlled by all the registers below $i$ ($i+1,...,n$) it is easy to see that we obtain exactly the right factors in (4), just in reverse order.

In [3]:
#input x list in bit format
def qft(x):
    n = len(x)
    c = cirq.Circuit()
    ##initialise n qubits
    q_l = cirq.LineQubit.range(n)
    # bring qubits into state x
    c.append([cirq.X(q_l[i]) for i in range(n) if x[i]])
    # add H gates and add phase gates note that here the phase of z is \pi hence the 
    #index is shifted by one e.g. R_2 corresponds to 2**{-1}
    for i in range(n):
        c.append([cirq.H(q_l[i])])
        c.append([(cirq.Z(q_l[i])**(2**(-(j-i)))).controlled_by(q_l[j]) for j in range(i+1,n)])
    return c
### fix order 

In [5]:
qft([0,1,1])

┌──┐
0: ───H───S────T─────────────
          │    │
1: ───X───@────┼H────S───────
               │     │
2: ───X────────@─────@───H───
              └──┘

In [97]:
s=cirq.Simulator()
c = qft([0,1,1])

In [98]:
s.simulate(c)

measurements: (no measurements)
output vector: 0.354|000⟩ - 0.354|001⟩ - 0.354j|010⟩ + 0.354j|011⟩ + (-0.25+0.25j)|100⟩ + (0.25-0.25j)|101⟩ + (0.25+0.25j)|110⟩ + (-0.25-0.25j)|111⟩

In [89]:
import numpy as np

In [100]:
for y in range(0,2**3):
    print(bin(y))
    print(np.exp(2*np.pi*1j*11*y/2**3)/np.sqrt(2**3))

0b0
(0.35355339059327373+0j)
0b1
(-0.2499999999999997+0.2500000000000003j)
0b10
(-8.661748989382706e-16-0.35355339059327373j)
0b11
(0.2500000000000009+0.24999999999999906j)
0b100
(-0.35355339059327373+1.7323497978765411e-15j)
0b101
(0.24999999999999847-0.2500000000000015j)
0b110
(2.5985246968148118e-15+0.35355339059327373j)
0b111
(-0.2500000000000021-0.24999999999999784j)


In [1]:
#Note that we have to relabel as mentoined earlier, e.g. |100> <-> 001 etc.
#This is to follow Ch.7, one could of course also just implement the circuit "upside down"